In [1]:
from marshmallow_jsonapi import fields
from marshmallow.validate import OneOf
from marshmallow_jsonapi.flask import Schema
from devtools import debug
from typing import Iterable

In [2]:
NOT_POSITIVE_NUMBER = 'not_positive_number'
INVALID_TEXT_FIELD = 'invalid_text_field'

FIELD_VALUE_PLACEHOLDER = 'field_value_placeholder'

validation_messages = {
    NOT_POSITIVE_NUMBER: f"'{FIELD_VALUE_PLACEHOLDER}' is not a positive number.",
    INVALID_TEXT_FIELD: f"'{FIELD_VALUE_PLACEHOLDER}' is not a valid %s."
}


class TextField(fields.Str):
    """
    String based field to be used in CSV schemas for data files validation. Uses OneOf validator against the input
    value. Sets default values for most used attributes.
    """
    def __init__(self, choices: Iterable, column_name: str, error_msg=None, *args, **kwargs, ):
        """
        Parameters
        ----------
        choices
            a sequence of strings which are valid values for this field.
        """
        error_msg = error_msg if error_msg else validation_messages[INVALID_TEXT_FIELD] % column_name
        super().__init__(
            validate=OneOf(choices=choices, error=error_msg),
            required = kwargs.get('required', True),
            allow_none = kwargs.get('allow_none', True),
            default = kwargs.get('default', ''),
            error_messages = {
                'invalid': error_msg,
                'validator_failed': error_msg
            },
            *args, 
            **kwargs
        )


class DurationField(fields.Float):
    """
    A positive floating point field with an appropriate validation error messages.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.validate = lambda x: x >= 0
        self.error_messages = {
            'invalid': validation_messages[NOT_POSITIVE_NUMBER],
            'validator_failed': validation_messages[NOT_POSITIVE_NUMBER]
        }


In [3]:
choices = ['Adrian', 'Balboa']

In [5]:
class MySchema(Schema):
    class Meta:
        type_ = 'users'
    id = fields.Str(missing=1)
    name = TextField(choices=choices, column_name='Name')
    

In [6]:
schema = MySchema()

In [7]:
debug(schema.fields)

<ipython-input-7-b37c8a997d94>:1 <module>
    schema.fields: {
        'id': (
            <fields.String(default=<marshmallow.missing>, attribute=None, validate=None, required=False,
            load_only=False, dump_only=False, missing=1, allow_none=False, error_messages={'required': 'Missing data for
            required field.', 'null': 'Field may not be null.', 'validator_failed': 'Invalid value.', 'invalid': 'Not a
            valid string.', 'invalid_utf8': 'Not a valid utf-8 string.'})>
        ),
        'name': (
            <fields.TextField(default='', attribute=None, validate=<OneOf(choices=['Adrian', 'Balboa'], labels=[],
            error="'field_value_placeholder' is not a valid Name.")>, required=True, load_only=False, dump_only=False,
            missing=<marshmallow.missing>, allow_none=True, error_messages={'required': 'Missing data for required
            field.', 'null': 'Field may not be null.', 'validator_failed': "'field_value_placeholder' is not a valid
     

In [10]:
good_data = {
    'data': {
        'type': 'users',
         'id': '12', 
         'attributes': {'name': 'Balboa'}
        }
}

In [11]:
schema.load(good_data)

/home/tomasz_kluczkowski/Dev/products_API/venv/lib/python3.6/site-packages/marshmallow_jsonapi/schema.py:239 _do_load
    data: {
        'data': {
            'type': 'users',
            'id': '12',
            'attributes': {
                'name': 'Balboa',
            },
        },
    } (dict) len=1
/home/tomasz_kluczkowski/Dev/products_API/venv/lib/python3.6/site-packages/marshmallow_jsonapi/schema.py:194 unwrap_item
    payload: {
        'id': '12',
        'name': 'Balboa',
    } (dict) len=2
/home/tomasz_kluczkowski/Dev/products_API/venv/lib/python3.6/site-packages/marshmallow_jsonapi/schema.py:264 _do_load
    result: {
        'id': '12',
        'name': 'Balboa',
    } (dict) len=2


{'id': '12', 'name': 'Balboa'}